In [30]:
import requests
import pandas as pd
from time import sleep
import numpy as np
from tqdm import tqdm

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [31]:
import pickle

In [37]:
# antes de empezar importamos las librerías que vamos a usar. 

import requests
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

# Importing pickle file

In [32]:
with open('../data/pickle/dataset_viral150_unique.pickle', 'rb') as tracks:
    tracks = pickle.load(tracks)

In [33]:
tracks.head()

,url,artist_clean,track_clean
8246,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert)
8247,https://open.spotify.com/track/343YBumqHu19cGo...,drake,fake love
8248,https://open.spotify.com/track/5aAx2yezTd8zXrk...,"the weeknd, daft punk",starboy
8249,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,"the chainsmokers, halsey",closer
8250,https://open.spotify.com/track/6fujklziTHa8uoM...,"rae sremmurd, gucci mane",black beatles


In [34]:
tracks.rename(columns={'track_clean':'track','artist_clean':'artist'}, inplace=True)
tracks.head()

,url,artist,track
8246,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert)
8247,https://open.spotify.com/track/343YBumqHu19cGo...,drake,fake love
8248,https://open.spotify.com/track/5aAx2yezTd8zXrk...,"the weeknd, daft punk",starboy
8249,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,"the chainsmokers, halsey",closer
8250,https://open.spotify.com/track/6fujklziTHa8uoM...,"rae sremmurd, gucci mane",black beatles


# Selenium

Configuro mi pestaña de selenium

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://chromedriver.chromium.org/downloads')

Voy a probar que me funcione la búsqueda de un elemento

In [ ]:
## iniciamos el web driver
opciones= Options()
opciones.add_argument("--start-maximized")
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://genius.com/')
driver.maximize_window()
driver.set_window_size(1920, 1080)
## te metes dentro de la página y con inspeccionar, seleccionas el elemento de las cookies --> copy / selector
##por si tu ordenador es un poco lento, le puedes poner un tiempo para que intente hacerlo
driver.implicitly_wait(30)
## si quieres tardar más, le pones un sleep
## sleep(20)
## acepto los cookies
driver.find_element('css selector', '#onetrust-accept-btn-handler').click()
sleep(1)
## busco el nombre del actor que quiero buscar
driver.find_element('css selector', '#application > div > div.PageHeaderdesktop__Container-bhx5ui-0.dmNhEr > div.PageHeaderdesktop__Left-bhx5ui-2.ctcvqz > form > input').send_keys('drake fake love', Keys.ENTER)
sleep(1)
driver.find_element('xpath', '/html/body/routable-page/ng-outlet/search-results-page/div/div[2]/div[1]/div[1]/search-result-section/div/div[2]/search-result-items/div/search-result-item/div/mini-song-card/a').click()
driver.implicitly_wait(30)
## extraigo la información de la letra de cada canción
lyrics = driver.find_element('css selector', '#lyrics-root > div:nth-child(3)').text
lyrics2 = driver.find_element('css selector', '#lyrics-root > div:nth-child(6)').text
sleep(1)
release_date = driver.find_element('css selector', '#application > main > div.SongPageGriddesktop-sc-1px5b71-0.jecoie.SongHeaderdesktop__Container-sc-1effuo1-0.fqavsD > div.SongHeaderdesktop__Column-sc-1effuo1-1.SongHeaderdesktop__Right-sc-1effuo1-4.ePANpp > div > div.HeaderMetadata__Grid-sc-1p42fnf-1.ePueMf > div:nth-child(2) > div').text
#application > main > div.SongPageGriddesktop-sc-1px5b71-0.jecoie.SongHeaderdesktop__Container-sc-1effuo1-0.kEkpYM > div.SongHeaderdesktop__Column-sc-1effuo1-1.SongHeaderdesktop__Right-sc-1effuo1-4.ePANpp > div > div.HeaderMetadata__Grid-sc-1p42fnf-1.ePueMf > div:nth-child(3) > div

In [ ]:
release_date

In [ ]:
lyrics + lyrics2

In [35]:
canciones = {
    'lyrics': [],
    'release_date': []
}

In [36]:
for index, row in tracks.iterrows():
    print(row['artist'].split(',')[0],'---', row['track'])
    ## iniciamos el web driver
    opciones= Options()
    opciones.add_argument("--start-maximized")
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get('https://genius.com/')
    driver.maximize_window()
    driver.set_window_size(1920, 1080)
    ## te metes dentro de la página y con inspeccionar, seleccionas el elemento de las cookies --> copy / selector
    ##por si tu ordenador es un poco lento, le puedes poner un tiempo para que intente hacerlo
    driver.implicitly_wait(30)
    ## si quieres tardar más, le pones un sleep
    ## sleep(20)
    ## acepto los cookies
    driver.find_element('css selector', '#onetrust-accept-btn-handler').click()
    sleep(1)
    ## busco el nombre del actor que quiero buscar
    driver.find_element('css selector', '#application > div > div.PageHeaderdesktop__Container-bhx5ui-0.dmNhEr > div.PageHeaderdesktop__Left-bhx5ui-2.ctcvqz > form > input').send_keys(row['artist'].split(',')[0] + ' ' + row['track'], Keys.ENTER)
    sleep(1)
    driver.find_element('xpath', '/html/body/routable-page/ng-outlet/search-results-page/div/div[2]/div[1]/div[1]/search-result-section/div/div[2]/search-result-items/div/search-result-item/div/mini-song-card/a').click()
    driver.implicitly_wait(30)
    ## extraigo la información de la letra de cada canción
    all_lyrics = ''
    for item in range(1,15):
        try:
            lyrics = driver.find_element('css selector', f'#lyrics-root > div:nth-child({item})').text
            if len(lyrics) > 12:
                all_lyrics += lyrics
            else:
                continue
        except:
            continue
    canciones['lyrics'].append(all_lyrics)
    sleep(1)
    try:
        release_date = driver.find_element('xpath', '//*[@id="application"]/main/div[1]/div[3]/div[1]/div[2]/div/div[1]/span[1]/span').text
    except:
        release_date = driver.find_element('xpath', '//*[@id="application"]/main/div[1]/div[4]/div/div[1]/div[3]/div').text
    canciones['release_date'].append(release_date)

migos --- bad and boujee (feat. lil uzi vert)


[WDM] - Downloading: 100%|██████████| 6.58M/6.58M [00:00<00:00, 35.7MB/s]


drake --- fake love
the weeknd --- starboy
the chainsmokers --- closer
rae sremmurd --- black beatles
shelley fka dram --- broccoli (feat. lil yachty)
drake --- one dance
aminé --- caroline
dj snake --- let me love you
big sean --- bounce back
the weeknd --- i feel it coming
bruno mars --- 24k magic
machine gun kelly --- bad things (with camila cabello)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="application"]/main/div[1]/div[4]/div/div[1]/div[3]/div"}
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x00C0F243]
	(No symbol) [0x00B97FD1]
	(No symbol) [0x00A8D04D]
	(No symbol) [0x00ABC0B0]
	(No symbol) [0x00ABC22B]
	(No symbol) [0x00AEE612]
	(No symbol) [0x00AD85D4]
	(No symbol) [0x00AEC9EB]
	(No symbol) [0x00AD8386]
	(No symbol) [0x00AB163C]
	(No symbol) [0x00AB269D]
	GetHandleVerifier [0x00EA9A22+2655074]
	GetHandleVerifier [0x00E9CA24+2601828]
	GetHandleVerifier [0x00CB8C0A+619850]
	GetHandleVerifier [0x00CB7830+614768]
	(No symbol) [0x00BA05FC]
	(No symbol) [0x00BA5968]
	(No symbol) [0x00BA5A55]
	(No symbol) [0x00BB051B]
	BaseThreadInitThunk [0x76F47D69+25]
	RtlInitializeExceptionChain [0x77C2BB9B+107]
	RtlClearBits [0x77C2BB1F+191]


In [ ]:
canciones

# Beautiful Soup
Prueba

In [38]:
canciones = {
    'lyrics': [],
    'release_date': []
}

In [54]:
url = 'https://genius.com/Drake-one-dance-lyrics'

In [55]:
res = requests.get(url)
res ## tiene que dar 200 

<Response [200]>

In [56]:
type(res.content) ## para consultarel html de la página

bytes

In [57]:
sopa = BeautifulSoup(res.content, 'html.parser')
type(sopa) ## es más amigable, pero sigue siendo html

bs4.BeautifulSoup

Datos canciones

In [59]:
pagina = sopa.text
re.findall(r'Date(\w+ \d+, \d+)', pagina)[0]

'April 5, 2016'

# Beautiful soup + selenium

In [73]:
canciones = {
    'lyrics': [],
    'release_date': []
}

In [76]:
for index, row in tracks.iterrows():
    print(row['artist'].split(',')[0],'---', row['track'])
    ## iniciamos el web driver
    opciones= Options()
    opciones.add_argument("--start-maximized")
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get('https://genius.com/')
    driver.maximize_window()
    driver.set_window_size(1920, 1080)
    ## te metes dentro de la página y con inspeccionar, seleccionas el elemento de las cookies --> copy / selector
    ##por si tu ordenador es un poco lento, le puedes poner un tiempo para que intente hacerlo
    driver.implicitly_wait(30)
    ## si quieres tardar más, le pones un sleep
    ## sleep(20)
    ## acepto los cookies
    driver.find_element('css selector', '#onetrust-accept-btn-handler').click()
    sleep(1)
    ## busco el nombre de canción que quiero buscar
    driver.find_element('css selector', '#application > div > div.PageHeaderdesktop__Container-bhx5ui-0.dmNhEr > div.PageHeaderdesktop__Left-bhx5ui-2.ctcvqz > form > input').send_keys(row['artist'].split(',')[0] + ' ' + row['track'], Keys.ENTER)
    sleep(1)
    ## selecciono el resultado de la búsqueda que quiero
    driver.find_element('xpath', '/html/body/routable-page/ng-outlet/search-results-page/div/div[2]/div[1]/div[2]/search-result-section/div/div[2]/search-result-items/div[1]/search-result-item/div/mini-song-card/a').click()
    driver.implicitly_wait(30)
    ## extraigo la información de la letra de cada canción
    page_source = driver.page_source
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')
    pagina = sopa.text
    canciones['release_date'].append(re.findall(r'Date(\w+ \d+, \d+)', pagina)[0])
    #break

migos --- bad and boujee (feat. lil uzi vert)
drake --- fake love
the weeknd --- starboy
the chainsmokers --- closer
rae sremmurd --- black beatles
shelley fka dram --- broccoli (feat. lil yachty)
drake --- one dance
aminé --- caroline
dj snake --- let me love you
big sean --- bounce back
the weeknd --- i feel it coming
bruno mars --- 24k magic
machine gun kelly --- bad things (with camila cabello)
21 savage --- x (feat. future)
zayn --- i don’t wanna live forever (fifty shades darker) - from "fifty shades darker (original motion picture soundtrack)"
maroon 5 --- don't wanna know
rob $tone --- chill bill (feat. j. davi$ & spooks)
j. cole --- deja vu
young m.a --- ooouuu
the weeknd --- party monster
chance the rapper --- no problem (feat. lil wayne & 2 chainz)
21 savage --- no heart
hailee steinfeld --- starving
the chainsmokers --- don't let me down (feat. daya)
ariana grande --- side to side
shawn mendes --- treat you better
martin garrix --- in the name of love
drake --- sneakin’
jus

In [ ]:
canciones

{'lyrics': [], 'release_date': ['April 5, 2016']}

In [61]:
tracks

,url,artist,track
8246,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert)
8247,https://open.spotify.com/track/343YBumqHu19cGo...,drake,fake love
8248,https://open.spotify.com/track/5aAx2yezTd8zXrk...,"the weeknd, daft punk",starboy
8249,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,"the chainsmokers, halsey",closer
8250,https://open.spotify.com/track/6fujklziTHa8uoM...,"rae sremmurd, gucci mane",black beatles
...,...,...,...
26131455,https://open.spotify.com/track/6jDOs0wGlRn4jCg...,fase yoda,butterflies
26131463,https://open.spotify.com/track/7eQHxigpuDJjCG5...,"max, ali gatie",butterflies
26131464,https://open.spotify.com/track/0kgRRHa5CMeCciI...,"cooper alan, rvshvd",colt 45 (country remix)
26131472,https://open.spotify.com/track/7GLqR9ToJLb0PV3...,kali,mmm mmm (feat. atl jacob)


In [67]:
urls = []
for index, row in tracks.iterrows():
    urls.append(f"https://genius.com/{row['artist'].split(',')[0].replace(' ','-')}-{row['track'].replace(' ','-')}")
urls[10:15]

['https://genius.com/the-weeknd-i-feel-it-coming',
 'https://genius.com/bruno-mars-24k-magic',
 'https://genius.com/machine-gun-kelly-bad-things-(with-camila-cabello)',
 'https://genius.com/21-savage-x-(feat.-future)',
 'https://genius.com/zayn-i-don’t-wanna-live-forever-(fifty-shades-darker)---from-"fifty-shades-darker-(original-motion-picture-soundtrack)"']

# Export

In [ ]:
with open('../data/pickle/genius_dataset.pickle', 'wb') as data_spotify:
    pickle.dump(tracks, data_spotify)